# Real World Application

## Packages

In [1]:
import pandas as pd
import numpy as np
import math
import torch
from matplotlib import pyplot as plt
import time

# GPytorch
import gpytorch
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, MaternKernel, InducingPointKernel
from gpytorch.distributions import MultivariateNormal
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy

# GPBoost
import gpboost as gpb

# DKL-GP
import viva
from viva import VIVACpp as VIVA, my_train_cpp as my_train

# Train-Test-Split
from sklearn.model_selection import train_test_split, KFold

# Batch
from torch.utils.data import TensorDataset, DataLoader

# Notebook
import tqdm
import tqdm.notebook as tqdm

# For CRPS & Log-Score
from scipy.stats import norm
from scipy.special import erf




## Functions

In [2]:
# Scale
def Scale(X_train, X_test, y_train, y_test):
    mins = X_train.min(0)
    maxs = X_train.max(0)
    X_train = (X_train-mins)/(maxs-mins)
    X_test = (X_test-mins)/(maxs-mins)

    mean = y_train.mean()
    sd = np.sqrt(y_train.var())

    y_train = (y_train-mean)/sd
    y_test = (y_test-mean)/sd
    
    return X_train, X_test, y_train, y_test

# Find too close points
def find_unique_X(X, threshold):
    n = len(X)
    mask = torch.ones(n, dtype=torch.bool)
    with torch.no_grad():
        for i in range(n - 1, 0, -1):
            if torch.cdist(X[i:i + 1], X[:i]).min() < threshold:
                mask[i] = False
    return mask

# CRPS
def crps_norm_vectorized(observations, means, sigmas):
    """
    Compute CRPS for multiple normal distributions.
    
    Parameters:
    observations (array-like): Observed values
    means (array-like): Means of the forecast distributions
    sigmas (array-like): Variance of the forecast distributions
    
    Returns:
    array-like: CRPS values
    """
    observations = np.asarray(observations)
    means = np.asarray(means)
    sigmas = np.sqrt(np.asarray(sigmas))
    
    z = (observations - means) / sigmas
    crps = sigmas * (z * (2 * norm.cdf(z) - 1) + 2 * norm.pdf(z) - 1 / np.sqrt(np.pi))
    return np.mean(crps)

# Log-Score
def log_score_norm_vectorized(observations, means, sigmas):
    """
    Compute Log Score for multiple normal distributions.
    
    Parameters:
    observations (array-like): Observed values
    means (array-like): Means of the forecast distributions
    sigmas (array-like): Variance of the forecast distributions
    
    Returns:
    array-like: Log Score values
    """
    observations = np.asarray(observations)
    means = np.asarray(means)
    sigmas = np.sqrt(np.asarray(sigmas))
    
    return np.mean(-norm.logpdf(observations, means, sigmas))

## Data

Choose dataset:

In [30]:
data = pd.read_csv('https://raw.githubusercontent.com/TimGyger/FSVA/refs/heads/main/Real%20World/Gaussian/Elevators/elevators.txt', sep=" ", header=None)

In [4]:
data.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,-0.041959,-294.2,-27.2010,-0.259920,-0.34449,0.35311,0.78963,23.5700,-0.000951,2.734000,0.00015,0.000054,0.000054,0.000057,0.000209,-1.807300e-08,0.000003,-3.012200e-08,0.000057
1,0.053351,456.8,8.7988,0.420080,-0.03449,0.65311,-4.21040,9.5698,0.003049,-0.066022,-0.00035,-0.000146,-0.000146,-0.000143,-0.000191,-1.807300e-08,0.000003,-3.012200e-08,-0.000143
2,0.329600,127.8,8.7988,0.070076,0.09551,-0.14689,-8.21040,3.5698,0.001049,-0.066022,-0.00065,-0.000646,-0.000646,-0.000643,0.000009,-1.807300e-08,0.000003,-3.012200e-08,-0.000643


In [5]:
# Features & Response
X = data.iloc[:, 1:]
y = data.iloc[:, 0]

X = X.values  # Selecting multiple features
y = y.values  # Selecting the target variable

d = X.shape[1] # Features

In [6]:
# Generate 5 disjoint test sets
kf = KFold(n_splits=5, shuffle=True, random_state=10)
test_indices_sets = []
train_indices_sets = []
for i, (_, test_indices) in enumerate(kf.split(X)):
    print(f"Fold {i+1} Test indices: {test_indices}")
    test_indices_sets.append(set(test_indices))
    train_indices_sets.append(set(range(len(X))) - test_indices_sets[i])


Fold 1 Test indices: [    0     7     9 ... 16583 16588 16597]
Fold 2 Test indices: [    5     8    22 ... 16593 16594 16598]
Fold 3 Test indices: [    6    13    17 ... 16587 16595 16596]
Fold 4 Test indices: [    1     4    11 ... 16585 16589 16591]
Fold 5 Test indices: [    2     3    10 ... 16577 16578 16590]


## Models

### Sparse Gaussian Process Regression (SGPR)

#### Parameters

In [7]:
num_inducing_points = 200
training_iterations = 100

#### Model

In [10]:
# SGPR
class SGPR(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(SGPR, self).__init__(train_x, train_y, likelihood)
        self.mean_module = ConstantMean()
        self.base_covar_module = ScaleKernel(MaternKernel(nu=3/2,ard_num_dims = d))
        self.covar_module = InducingPointKernel(self.base_covar_module, inducing_points=train_x[:num_inducing_points, :].clone(), likelihood=likelihood)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)



#### Training & Prediction

In [11]:
# Define row and column names
row_names = ['MAE', 'RMSE', 'CRPS', 'LS','Time']
col_names = ['Iteration 1', 'Iteration 2', 'Iteration 3', 'Iteration 4', 'Iteration 5']

# Create an empty DataFrame with the specified row and column names
dat_SGPR = pd.DataFrame(np.zeros((len(row_names), len(col_names))), 
                      index=row_names, columns=col_names)

for j in range(5):

    # Data
    X_test = np.array([X[idx] for idx in test_indices_sets[j]])
    X_train = np.array([X[idx] for idx in train_indices_sets[j]])
    y_test = np.array([y[idx] for idx in test_indices_sets[j]])
    y_train = np.array([y[idx] for idx in train_indices_sets[j]])
    X_train, X_test, y_train, y_test = Scale(X_train, X_test, y_train, y_test)
    print(X_train.shape)
    print(y_train.shape)
    
    # Convert the numpy arrays to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

    mask_all = find_unique_X(X_train_tensor, 0.01)
    X_train_tensor = X_train_tensor[mask_all]
    y_train_tensor = y_train_tensor[mask_all]
    print(X_train.shape)
    print(y_train.shape)
    
    print(f"Split {i+1} Test Response: {y_test}")
    
    # Model
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model_SGPR = SGPR(X_train_tensor, y_train_tensor, likelihood)
    # Use the adam optimizer
    optimizer = torch.optim.LBFGS(model_SGPR.parameters(), lr=0.1)
    if torch.cuda.is_available():
        model_SGPR = model_SGPR.cuda()
        likelihood = likelihood.cuda()
    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model_SGPR)
    
    # Train
    def train():
        iterator = tqdm.tqdm(range(training_iterations), desc="Train")
        print("Training started.")
        for i in iterator:
            def closure():
                optimizer.zero_grad()
                output = model_SGPR(X_train_tensor)
                loss = -mll(output, y_train_tensor)
                loss.backward()
                return loss

            loss = optimizer.step(closure)
            iterator.set_postfix(loss=loss.item())
        
            if (i+1) % 10 == 0:
                print(f"Iteration {i+1}/{training_iterations}")
                print(f"Loss: {loss.item()}\n")

        print("Training completed.")
    
    start_time = time.time()
    %time train()
    runtime_SGPR = time.time() - start_time

    # Prediction
    print("Prediction started.")
    model_SGPR.eval()
    likelihood.eval()
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        preds_SGPR = model_SGPR.likelihood(model_SGPR(X_test_tensor))
        variances_SGPR = preds_SGPR.variance
        MAE_SGPR = torch.mean(torch.abs(preds_SGPR.mean - y_test_tensor))
        RMSE_SGPR = torch.sqrt(torch.mean(torch.square(preds_SGPR.mean - y_test_tensor)))
        LS_SGPR = log_score_norm_vectorized(y_test_tensor,preds_SGPR.mean,variances_SGPR)
        CRPS_SGPR = crps_norm_vectorized(y_test_tensor,preds_SGPR.mean,variances_SGPR)
        dat_SGPR.loc[row_names[0], col_names[j]] = MAE_SGPR.item()
        dat_SGPR.loc[row_names[1], col_names[j]] = RMSE_SGPR.item()
        dat_SGPR.loc[row_names[2], col_names[j]] = CRPS_SGPR.item()
        dat_SGPR.loc[row_names[3], col_names[j]] = LS_SGPR.item()
        dat_SGPR.loc[row_names[4], col_names[j]] = runtime_SGPR
        dat_SGPR.to_csv('C:/Users/JumpStart/Desktop/IterativeFSAVecchia/dat_SGPR.txt', sep='\t', index=True)
        print('Test MAE: {}'.format(MAE_SGPR))
        print('Test RMSE: {}'.format(RMSE_SGPR))
        print('Test CRPS: {}'.format(CRPS_SGPR))
        print('Test LS: {}'.format(LS_SGPR))
        print('Runtime: {} seconds'.format(runtime_SGPR))
    

(13279, 18)
(13279,)
(13279, 18)
(13279,)
Split 5 Test Response: [-0.16737101  1.01674971 -0.58308711 ...  0.02513773  0.02513773
 -0.16737101]


Train:   0%|          | 0/100 [00:00<?, ?it/s]

Training started.
Iteration 10/100
Loss: 0.45497044920921326

Iteration 20/100
Loss: 0.4492957592010498

Iteration 30/100
Loss: 0.44652751088142395

Iteration 40/100
Loss: 0.4449770748615265

Iteration 50/100
Loss: 0.4439089596271515

Iteration 60/100
Loss: 0.44319167733192444

Iteration 70/100
Loss: 0.4426751136779785

Iteration 80/100
Loss: 0.44232964515686035

Iteration 90/100
Loss: 0.44208207726478577

Iteration 100/100
Loss: 0.44189199805259705

Training completed.
CPU times: total: 4min 7s
Wall time: 3min 42s
Prediction started.
Test MAE: 0.3194529414176941
Test RMSE: 0.4045885503292084
Test CRPS: 0.31869217568353053
Test LS: 81512.73155966705
Runtime: 222.30768465995789 seconds
(13279, 18)
(13279,)


C:\Users\JumpStart\anaconda3\envs\FSVA\Lib\site-packages\gpytorch\distributions\multivariate_normal.py:319: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-06.
  warnings.warn(


(13279, 18)
(13279,)
Split 5 Test Response: [ 0.0245169  -1.05702319 -0.16953456 ... -0.3735399   0.0245169
 -0.58858211]


...

Training started.
Iteration 10/100
Loss: 0.4553087651729584

Iteration 20/100
Loss: 0.4494631886482239

Iteration 30/100
Loss: 0.4470326602458954

Iteration 40/100
Loss: 0.44558340311050415

Iteration 50/100
Loss: 0.44475215673446655

Iteration 60/100
Loss: 0.44421815872192383

Iteration 70/100
Loss: 0.4438973665237427

Iteration 80/100
Loss: 0.443587988615036

Iteration 90/100
Loss: 0.44333207607269287

Iteration 100/100
Loss: 0.4431096315383911

Training completed.
CPU times: total: 3min 52s
Wall time: 3min 43s
Prediction started.
Test MAE: 0.29472801089286804
Test RMSE: 0.37721049785614014
Test CRPS: 0.29350261144920564
Test LS: 70664.52722647059
Runtime: 223.91444969177246 seconds
(13279, 18)
(13279,)
(13279, 18)
(13279,)
Split 5 Test Response: [ 0.20848863  0.86715903  0.71248529 ...  0.20848863 -0.58270519
 -0.36952906]


Train:   0%|          | 0/100 [00:00<?, ?it/s]

Training started.
Iteration 10/100
Loss: 0.46023061871528625

Iteration 20/100
Loss: 0.4533671736717224

Iteration 30/100
Loss: 0.45069336891174316

Iteration 40/100
Loss: 0.44920021295547485

Iteration 50/100
Loss: 0.44820746779441833

Iteration 60/100
Loss: 0.4475512206554413

Iteration 70/100
Loss: 0.4471484124660492

Iteration 80/100
Loss: 0.44681400060653687

Iteration 90/100
Loss: 0.4465155303478241

Iteration 100/100
Loss: 0.44638675451278687

Training completed.
CPU times: total: 3min 48s
Wall time: 3min 40s
Prediction started.
Test MAE: 0.5704613924026489
Test RMSE: 0.6599612832069397
Test CRPS: 0.5422630124245507
Test LS: 182516.06438921802
Runtime: 220.25845623016357 seconds
(13280, 18)
(13280,)
(13280, 18)
(13280,)
Split 5 Test Response: [-0.5778195  -0.3624234   1.32213445 ... -0.5778195  -0.3624234
  0.03628847]


Train:   0%|          | 0/100 [00:00<?, ?it/s]

Training started.
Iteration 10/100
Loss: 0.46179959177970886

Iteration 20/100
Loss: 0.45609545707702637

Iteration 30/100
Loss: 0.45309340953826904

Iteration 40/100
Loss: 0.45121192932128906

Iteration 50/100
Loss: 0.4501310884952545

Iteration 60/100
Loss: 0.4494316577911377

Iteration 70/100
Loss: 0.44885218143463135

Iteration 80/100
Loss: 0.44846832752227783

Iteration 90/100
Loss: 0.44817453622817993

Iteration 100/100
Loss: 0.4479650855064392

Training completed.
CPU times: total: 4min 19s
Wall time: 3min 54s
Prediction started.
Test MAE: 0.29595133662223816
Test RMSE: 0.39199134707450867
Test CRPS: 0.2184356232918123
Test LS: 0.5183541497854907
Runtime: 234.66342616081238 seconds


### Stochastic Variational Gaussian Processes (SVGP)

#### Parameters

In [27]:
num_inducing_points = 200
training_iterations = 1000

#### Model

In [28]:
class SVGP(ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(SVGP, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(MaternKernel(nu=3/2,ard_num_dims = d))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

#### Training & Prediction

In [29]:
# Define row and column names
row_names = ['MAE', 'RMSE', 'CRPS', 'LS','Time']
col_names = ['Iteration 1', 'Iteration 2', 'Iteration 3', 'Iteration 4', 'Iteration 5']

# Create an empty DataFrame with the specified row and column names
dat_SVGP = pd.DataFrame(np.zeros((len(row_names), len(col_names))), 
                      index=row_names, columns=col_names)

for j in range(5):

    # Data
    X_test = np.array([X[idx] for idx in test_indices_sets[j]])
    X_train = np.array([X[idx] for idx in train_indices_sets[j]])
    y_test = np.array([y[idx] for idx in test_indices_sets[j]])
    y_train = np.array([y[idx] for idx in train_indices_sets[j]])
    X_train, X_test, y_train, y_test = Scale(X_train, X_test, y_train, y_test)
    print(X_train.shape)
    print(y_train.shape)
    
    # Convert the numpy arrays to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
    # Remove close points
    mask_all = find_unique_X(X_train_tensor, 0.01)
    X_train_tensor = X_train_tensor[mask_all]
    y_train_tensor = y_train_tensor[mask_all]
    print(X_train.shape)
    print(y_train.shape)
    # Batches
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)
    
    print(f"Split {i+1} Test Response: {y_test}")
    
    # Model
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    inducing_points = X_train_tensor[:num_inducing_points, :]
    model_SVGP = SVGP(inducing_points=inducing_points)
    if torch.cuda.is_available():
        model_SVGP = model_SVGP.cuda()
        likelihood = likelihood.cuda()
    # Use the adam optimizer
    model_SVGP.train()
    likelihood.train()
    optimizer = torch.optim.Adam([
        {'params': model_SVGP.parameters()},
        {'params': likelihood.parameters()},
    ], lr=0.01)
    # Our loss object. We're using the VariationalELBO
    mll = gpytorch.mlls.VariationalELBO(likelihood, model_SVGP, num_data=y_train_tensor.size(0))
    
    # Train
    def train():
        print("Training started.")
        epochs_iter = tqdm.tqdm(range(training_iterations), desc="Epoch")
        for i in epochs_iter:
            # Within each iteration, we will go over each minibatch of data
            minibatch_iter = tqdm.tqdm(train_loader, desc="Minibatch", leave=False)
            for x_batch, y_batch in minibatch_iter:
                optimizer.zero_grad()
                output = model_SVGP(x_batch)
                loss = -mll(output, y_batch)
                minibatch_iter.set_postfix(loss=loss.item())
                loss.backward()
                optimizer.step()
            if (i+1) % 100 == 0:
                print(f"Iteration {i + 1}/{training_iterations}")
                print(f"Loss: {loss.item()}\n")
                
        print("Training completed.")
    
    start_time = time.time()
    %time train()
    runtime_SVGP = time.time() - start_time

    # Prediction
    print("Prediction started.")
    model_SVGP.eval()
    likelihood.eval()
    means_SVGP = torch.tensor([0.])
    variances_SVGP = torch.tensor([0.])
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            preds = model_SVGP(x_batch)
            means_SVGP = torch.cat([means_SVGP, preds.mean.cpu()])
            variances_SVGP = torch.cat([variances_SVGP, preds.variance.cpu()])
        means_SVGP = means_SVGP[1:]
        variances_SVGP = variances_SVGP[1:]
        
        MAE_SVGP = torch.mean(torch.abs(means_SVGP - y_test_tensor))
        RMSE_SVGP = torch.sqrt(torch.mean(torch.square(means_SVGP - y_test_tensor)))
        LS_SVGP = log_score_norm_vectorized(y_test_tensor,means_SVGP,variances_SVGP)
        CRPS_SVGP = crps_norm_vectorized(y_test_tensor,means_SVGP,variances_SVGP)
        dat_SVGP.loc[row_names[0], col_names[j]] = MAE_SVGP.item()
        dat_SVGP.loc[row_names[1], col_names[j]] = RMSE_SVGP.item()
        dat_SVGP.loc[row_names[2], col_names[j]] = CRPS_SVGP.item()
        dat_SVGP.loc[row_names[3], col_names[j]] = LS_SVGP.item()
        dat_SVGP.loc[row_names[4], col_names[j]] = runtime_SVGP
        dat_SVGP.to_csv('C:/Users/JumpStart/Desktop/IterativeFSAVecchia/dat_SVGP.txt', sep='\t', index=True)
        print('Test MAE: {}'.format(MAE_SVGP))
        print('Test RMSE: {}'.format(RMSE_SVGP))
        print('Test CRPS: {}'.format(CRPS_SVGP))
        print('Test LS: {}'.format(LS_SVGP))
        print('Runtime: {} seconds'.format(runtime_SVGP))

(13279, 18)
(13279,)
(13279, 18)
(13279,)
Split 5 Test Response: [-0.16737101  1.01674971 -0.58308711 ...  0.02513773  0.02513773
 -0.16737101]
Training started.


Epoch:   0%|          | 0/1000 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/13 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/13 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/13 [00:00<?, ?it/s]

...

Minibatch:   0%|          | 0/13 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/13 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/13 [00:00<?, ?it/s]

Minibatch:   0%|          | 0/13 [00:00<?, ?it/s]

Iteration 1000/1000
Loss: 0.5060689449310303

Training completed.
CPU times: total: 3min 43s
Wall time: 7min 21s
Prediction started.
Test MAE: 0.2857082784175873
Test RMSE: 0.38177815079689026
Test CRPS: 0.25394402214982653
Test LS: 21.263040974217876
Runtime: 441.42738342285156 seconds


### Double-Kullback-Leibler-optimal Gaussian-process approximation (DKL-GP)

In [9]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
classify = False
rho = 1.3
n_Epoch=50

In [10]:
X_tensor = torch.cat((X_train_tensor, X_test_tensor), dim=0)
y_tensor = torch.cat((y_train_tensor, y_test_tensor), dim=0)

In [11]:
# Kernel
K = gpytorch.kernels.ScaleKernel(MaternKernel(nu=3/2,ard_num_dims = d))

model_DKL = VIVA(X_tensor, y_tensor, K, likelihood, rho, n_test=y_test.size,
                             classify=classify, use_ic0=True)

C:\Users\JumpStart\anaconda3\envs\FSVA\Lib\site-packages\viva\utils.py:198: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\SparseCsrTensorImpl.cpp:55.)
  [n, n]).to_sparse_csr()


Maxmin ordering for n = 16599, d = 18, rho = 1.3 used 424.7689581000013 seconds
Laplacian initialization used 146.14467579999473 seconds


#### Train

In [12]:
start_time = time.time()
my_train(model_DKL, n_Epoch=n_Epoch)
runtime_DKL = time.time() - start_time

time_update_U = 1.3174385000602342, time_ELBO_fwd = 12.08061799989082, time_ELBO_bwd = 8.0548606000375, time_sp_trisolve = 0.8307720001321286, time_sub_matrix = 9.65840179999941 time_innerMean = 0.026686100027291104 time_logDet = 0.20178649999434128 time_expected_likelihood = 0.8375556000391953
Epoch 0 ELBO is -63044.0
time_update_U = 2.636361799959559, time_ELBO_fwd = 24.477065399929415, time_ELBO_bwd = 16.202766200061888, time_sp_trisolve = 1.667282500071451, time_sub_matrix = 19.57105130000855 time_innerMean = 0.05364330002339557 time_logDet = 0.4039504999818746 time_expected_likelihood = 1.7010210000153165
Epoch 1 ELBO is -54022.9453125
time_update_U = 3.9700169001007453, time_ELBO_fwd = 36.895887400140055, time_ELBO_bwd = 24.33566089987289, time_sp_trisolve = 2.4808278999698814, time_sub_matrix = 29.552868400060106 time_innerMean = 0.08035639999434352 time_logDet = 0.6042225001438055 time_expected_likelihood = 2.5542873998347204
Epoch 2 ELBO is -45641.171875
time_update_U = 5.3104

#### Prediction

In [13]:
preds_DKL, variances_DKL = model_DKL.predict()

MAE_DKL = torch.mean(torch.abs(preds_DKL - y_test_tensor))
RMSE_DKL = torch.sqrt(torch.mean(torch.square(preds_DKL - y_test_tensor)))
CRPS_DKL = crps_norm_vectorized(y_test_tensor,preds_DKL,variances_DKL)
LS_DKL = log_score_norm_vectorized(y_test_tensor,preds_DKL,variances_DKL)
    
print('Test MAE: {}'.format(MAE_DKL))
print('Test RMSE: {}'.format(RMSE_DKL))
print('Test CRPS: {}'.format(CRPS_DKL))
print('Test LS: {}'.format(LS_DKL))
print('Runtime: {} seconds'.format(runtime_DKL))


Test MAE: 0.34644871950149536
Test RMSE: 0.47799739241600037
Test CRPS: 0.2898806343316658
Test LS: 18.6326549131507
Runtime: 1102.8253371715546 seconds


### Vecchia

In [9]:
likelihood = "gaussian"
num_Vecchia_neighbors= 30

In [10]:
model_vecchia = gpb.GPModel(gp_coords=X_train, cov_function="matern_ard", cov_fct_shape=1.5,
                       likelihood=likelihood,vecchia_ordering = "random",num_neighbors =num_Vecchia_neighbors,
                       matrix_inversion_method = "cholesky", gp_approx="vecchia",seed = 10)

[GPBoost] [Info] Starting nearest neighbor search for Vecchia approximation
[GPBoost] [Info] Nearest neighbors for Vecchia approximation found


#### Train

In [11]:
model_vecchia.set_optim_params(params={"optimizer_cov": "lbfgs", "trace": True, "init_cov_pars": np.array([0.5, 0.5] + [0.1] * X_train.shape[1])})
start_time = time.time()
model_vecchia.fit(y=y_train);
runtime_Vecchia = time.time() - start_time

[GPBoost] [Debug] GPModel: initial parameters: 
[GPBoost] [Debug] cov_pars[0]: 0.5
[GPBoost] [Debug] cov_pars[1]: 0.5
[GPBoost] [Debug] cov_pars[2]: 0.1
[GPBoost] [Debug] cov_pars[3]: 0.1
[GPBoost] [Debug] cov_pars[4]: 0.1
[GPBoost] [Debug] cov_pars[5]: 0.1
[GPBoost] [Debug] cov_pars[6]: 0.1
[GPBoost] [Debug] cov_pars[7]: 0.1
[GPBoost] [Debug] cov_pars[8]: 0.1
[GPBoost] [Debug] cov_pars[9]: 0.1
[GPBoost] [Debug] cov_pars[10]: 0.1
[GPBoost] [Debug] cov_pars[11]: 0.1
[GPBoost] [Debug] cov_pars[12]: 0.1
[GPBoost] [Debug] cov_pars[13]: 0.1
[GPBoost] [Debug] cov_pars[14]: 0.1
[GPBoost] [Debug] cov_pars[15]: 0.1
[GPBoost] [Debug] cov_pars[16]: 0.1
[GPBoost] [Debug] cov_pars[17]: 0.1
[GPBoost] [Debug] cov_pars[18]: 0.1
[GPBoost] [Debug] cov_pars[19]: 0.1
[GPBoost] [Info] start
[GPBoost] [Info]  time until vecchia = 0.503977 
[GPBoost] [Info] CalcCovFactorVecchia
[GPBoost] [Info] start
[GPBoost] [Info]  time until0 = 0.104817 
[GPBoost] [Info]  time until = 0.745665 
[GPBoost] [Info] grad_cov_

#### Prediction

In [41]:
pred_Vecchia = model_vecchia.predict(gp_coords_pred=X_test, predict_var=True)

MAE_Vecchia = np.mean(np.abs(pred_Vecchia['mu'] - y_test))
RMSE_Vecchia = np.sqrt(np.mean(np.square(pred_Vecchia['mu'] - y_test)))
CRPS_Vecchia = crps_norm_vectorized(y_test,pred_Vecchia['mu'],pred_Vecchia['var'])
LS_Vecchia = log_score_norm_vectorized(y_test,pred_Vecchia['mu'],pred_Vecchia['var'])
print('Test MAE: {}'.format(MAE_Vecchia))
print('Test RMSE: {}'.format(RMSE_Vecchia))
print('Test CRPS: {}'.format(CRPS_Vecchia))
print('Test LS: {}'.format(LS_Vecchia))
print('Runtime: {} seconds'.format(runtime_Vecchia))

NameError: name 'model_vecchia' is not defined

### FITC

In [13]:
likelihood = "gaussian"
num_inducing_points= 200

In [14]:
model_fitc = gpb.GPModel(gp_coords=X_train, cov_function="matern_ard", cov_fct_shape=1.5,
                       likelihood=likelihood,num_ind_points = num_inducing_points,ind_points_selection = "kmeans++",
                       matrix_inversion_method = "cholesky", gp_approx="fitc",seed = 10)

#### Train

In [15]:
model_fitc.set_optim_params(params={"optimizer_cov": "lbfgs", "trace": True})
start_time = time.time()
model_fitc.fit(y=y_train);
runtime_fitc = time.time() - start_time

[GPBoost] [Debug] GPModel: initial parameters: 
[GPBoost] [Debug] cov_pars[0]: 0.500038
[GPBoost] [Debug] cov_pars[1]: 0.500038
[GPBoost] [Debug] cov_pars[2]: 0.0262892
[GPBoost] [Debug] cov_pars[3]: 0.0243421
[GPBoost] [Debug] cov_pars[4]: 0.0263923
[GPBoost] [Debug] cov_pars[5]: 0.0198357
[GPBoost] [Debug] cov_pars[6]: 0.0323468
[GPBoost] [Debug] cov_pars[7]: 0.0428232
[GPBoost] [Debug] cov_pars[8]: 0.0188912
[GPBoost] [Debug] cov_pars[9]: 0.0208238
[GPBoost] [Debug] cov_pars[10]: 0.00617098
[GPBoost] [Debug] cov_pars[11]: 0.0244012
[GPBoost] [Debug] cov_pars[12]: 0.024828
[GPBoost] [Debug] cov_pars[13]: 0.0248297
[GPBoost] [Debug] cov_pars[14]: 0.0256975
[GPBoost] [Debug] cov_pars[15]: 0.00940595
[GPBoost] [Debug] cov_pars[16]: 3.23264e-05
[GPBoost] [Debug] cov_pars[17]: 0.00648926
[GPBoost] [Debug] cov_pars[18]: 0.000113293
[GPBoost] [Debug] cov_pars[19]: 0.0257038
[GPBoost] [Info] start
[GPBoost] [Info]  time until ip = 0.634819 
[GPBoost] [Info]  time until vecchia = 0 
[GPBoost]

#### Prediction

In [17]:
pred_fitc = model_fitc.predict(gp_coords_pred=X_test, predict_var=True)

MAE_fitc = np.mean(np.abs(pred_fitc['mu'] - y_test))
RMSE_fitc = np.sqrt(np.mean(np.square(pred_fitc['mu'] - y_test)))
CRPS_fitc = crps_norm_vectorized(y_test,pred_fitc['mu'],pred_fitc['var'])
LS_fitc = log_score_norm_vectorized(y_test,pred_fitc['mu'],pred_fitc['var'])
print('Test MAE: {}'.format(MAE_fitc))
print('Test RMSE: {}'.format(RMSE_fitc))
print('Test CRPS: {}'.format(CRPS_fitc))
print('Test LS: {}'.format(LS_fitc))
print('Runtime: {} seconds'.format(runtime_fitc))

[GPBoost] [Info] a1 6471.63 6400.3 6470.21
Test MAE: 0.29718516814169615
Test RMSE: 0.3866821507389262
Test CRPS: 0.30741404442912323
Test LS: 2.165376906392396
Runtime: 555.664350271225 seconds


### FSVA

In [14]:
likelihood = "gaussian"
num_inducing_points= 200
num_Vecchia_neighbors = 30

In [15]:
# correlation based Vecchia neighbor search
model_FSVA_corr = gpb.GPModel(gp_coords=X_train, cov_function="matern_ard", cov_fct_shape=1.5,
                       likelihood=likelihood,vecchia_ordering = "random",num_neighbors =num_Vecchia_neighbors,
                       num_ind_points = num_inducing_points,ind_points_selection = "kmeans++",       
                       matrix_inversion_method = "cholesky", gp_approx="vecchia",seed = 4)
# Euclidean based Vecchia neighbor search
model_FSVA_eucl = gpb.GPModel(gp_coords=X_train, cov_function="matern_ard", cov_fct_shape=1.5,
                       likelihood=likelihood,vecchia_ordering = "random",num_neighbors =num_Vecchia_neighbors,
                       num_ind_points = num_inducing_points,ind_points_selection = "kmeans++",       
                       matrix_inversion_method = "cholesky", gp_approx="vecchia",seed = 2)

[GPBoost] [Info] Starting nearest neighbor search for Vecchia approximation
[GPBoost] [Info] Nearest neighbors for Vecchia approximation found
[GPBoost] [Info] Starting nearest neighbor search for Vecchia approximation
[GPBoost] [Info] Nearest neighbors for Vecchia approximation found


#### Train

In [16]:
model_FSVA_corr.set_optim_params(params={"optimizer_cov": "lbfgs", "trace": True})
start_time = time.time()
model_FSVA_corr.fit(y=y_train);
runtime_FSVA_corr = time.time() - start_time

[GPBoost] [Debug] GPModel: initial parameters: 
[GPBoost] [Debug] cov_pars[0]: 0.500038
[GPBoost] [Debug] cov_pars[1]: 0.500038
[GPBoost] [Debug] cov_pars[2]: 0.0262468
[GPBoost] [Debug] cov_pars[3]: 0.025148
[GPBoost] [Debug] cov_pars[4]: 0.0264143
[GPBoost] [Debug] cov_pars[5]: 0.0206516
[GPBoost] [Debug] cov_pars[6]: 0.0321802
[GPBoost] [Debug] cov_pars[7]: 0.0432686
[GPBoost] [Debug] cov_pars[8]: 0.0189276
[GPBoost] [Debug] cov_pars[9]: 0.0209103
[GPBoost] [Debug] cov_pars[10]: 0.00631064
[GPBoost] [Debug] cov_pars[11]: 0.026014
[GPBoost] [Debug] cov_pars[12]: 0.0264779
[GPBoost] [Debug] cov_pars[13]: 0.0265268
[GPBoost] [Debug] cov_pars[14]: 0.0275006
[GPBoost] [Debug] cov_pars[15]: 0.00945416
[GPBoost] [Debug] cov_pars[16]: 0.000921304
[GPBoost] [Debug] cov_pars[17]: 0.00642279
[GPBoost] [Debug] cov_pars[18]: 0.00186243
[GPBoost] [Debug] cov_pars[19]: 0.0276043
[GPBoost] [Info] start
[GPBoost] [Info]  time until ip = 0.644678 
[GPBoost] [Info]  time until CoverTrees for kNN = 0.2

In [17]:
model_FSVA_eucl.set_optim_params(params={"optimizer_cov": "lbfgs", "trace": True})
start_time = time.time()
model_FSVA_eucl.fit(y=y_train);
runtime_FSVA_eucl = time.time() - start_time

[GPBoost] [Debug] GPModel: initial parameters: 
[GPBoost] [Debug] cov_pars[0]: 0.500038
[GPBoost] [Debug] cov_pars[1]: 0.500038
[GPBoost] [Debug] cov_pars[2]: 0.0262468
[GPBoost] [Debug] cov_pars[3]: 0.025148
[GPBoost] [Debug] cov_pars[4]: 0.0264143
[GPBoost] [Debug] cov_pars[5]: 0.0206516
[GPBoost] [Debug] cov_pars[6]: 0.0321802
[GPBoost] [Debug] cov_pars[7]: 0.0432686
[GPBoost] [Debug] cov_pars[8]: 0.0189276
[GPBoost] [Debug] cov_pars[9]: 0.0209103
[GPBoost] [Debug] cov_pars[10]: 0.00631064
[GPBoost] [Debug] cov_pars[11]: 0.026014
[GPBoost] [Debug] cov_pars[12]: 0.0264779
[GPBoost] [Debug] cov_pars[13]: 0.0265268
[GPBoost] [Debug] cov_pars[14]: 0.0275006
[GPBoost] [Debug] cov_pars[15]: 0.00945416
[GPBoost] [Debug] cov_pars[16]: 0.000921304
[GPBoost] [Debug] cov_pars[17]: 0.00642279
[GPBoost] [Debug] cov_pars[18]: 0.00186243
[GPBoost] [Debug] cov_pars[19]: 0.0276043
[GPBoost] [Info] start
[GPBoost] [Info]  time until ip = 0.669085 
[GPBoost] [Info]  time until vecchia = 0.405371 
[GPB

Exception ignored on calling ctypes callback function: <function _log_callback at 0x0000027DEB184540>
Traceback (most recent call last):
  File "C:\Users\JumpStart\anaconda3\envs\FSVA\Lib\site-packages\gpboost\basic.py", line 112, in _log_callback
    def _log_callback(msg):
    
KeyboardInterrupt: 


 time until = 4.02642 
[GPBoost] [Info] grad_cov_aux_par[0]: -14.5643
[GPBoost] [Info] grad_cov_aux_par[1]: 19.7521
[GPBoost] [Info] grad_cov_aux_par[2]: 19.3946
[GPBoost] [Info] grad_cov_aux_par[3]: 16.9925
[GPBoost] [Info] grad_cov_aux_par[4]: 8.27964
[GPBoost] [Info] grad_cov_aux_par[5]: 15.7094
[GPBoost] [Info] grad_cov_aux_par[6]: 6.83883
[GPBoost] [Info] grad_cov_aux_par[7]: 13.4888
[GPBoost] [Info] grad_cov_aux_par[8]: 10.9945
[GPBoost] [Info] grad_cov_aux_par[9]: 10.3133
[GPBoost] [Info] grad_cov_aux_par[10]: 3.74571
[GPBoost] [Info] grad_cov_aux_par[11]: 2.17182
[GPBoost] [Info] grad_cov_aux_par[12]: 2.1638
[GPBoost] [Info] grad_cov_aux_par[13]: 2.09244
[GPBoost] [Info] grad_cov_aux_par[14]: 0.0192275
[GPBoost] [Info] grad_cov_aux_par[15]: 0
[GPBoost] [Info] grad_cov_aux_par[16]: 6.89182e-05
[GPBoost] [Info] grad_cov_aux_par[17]: 3.30034e-41
[GPBoost] [Info] grad_cov_aux_par[18]: 2.07683
[GPBoost] [Debug] Initial approximate negative marginal log-likelihood: 18810.9
[GPBoost] 

#### Test

In [18]:
pred_FSVA_corr = model_FSVA_corr.predict(gp_coords_pred=X_test, predict_var=True)

MAE_FSVA_corr = np.mean(np.abs(pred_FSVA_corr['mu'] - y_test))
RMSE_FSVA_corr = np.sqrt(np.mean(np.square(pred_FSVA_corr['mu'] - y_test)))
CRPS_FSVA_corr = crps_norm_vectorized(y_test,pred_FSVA_corr['mu'],pred_FSVA_corr['var'])
LS_FSVA_corr = log_score_norm_vectorized(y_test,pred_FSVA_corr['mu'],pred_FSVA_corr['var'])
print('Test MAE: {}'.format(MAE_FSVA_corr))
print('Test RMSE: {}'.format(RMSE_FSVA_corr))
print('Test CRPS: {}'.format(CRPS_FSVA_corr))
print('Test LS: {}'.format(LS_FSVA_corr))
print('Runtime: {} seconds'.format(runtime_FSVA_corr))

[GPBoost] [Info]  time until CoverTrees for kNN = 0.259291 
[GPBoost] [Info] num threads 12 16599 1022
Test MAE: 0.2723539323245213
Test RMSE: 0.3580914667281354
Test CRPS: 0.2242406112597114
Test LS: 2.754282104908586
Runtime: 3042.034034729004 seconds


In [19]:
pred_FSVA_eucl = model_FSVA_eucl.predict(gp_coords_pred=X_test, predict_var=True)

MAE_FSVA_eucl = np.mean(np.abs(pred_FSVA_eucl['mu'] - y_test))
RMSE_FSVA_eucl = np.sqrt(np.mean(np.square(pred_FSVA_eucl['mu'] - y_test)))
CRPS_FSVA_eucl = crps_norm_vectorized(y_test,pred_FSVA_eucl['mu'],pred_FSVA_eucl['var'])
LS_FSVA_eucl = log_score_norm_vectorized(y_test,pred_FSVA_eucl['mu'],pred_FSVA_eucl['var'])
print('Test MAE: {}'.format(MAE_FSVA_eucl))
print('Test RMSE: {}'.format(RMSE_FSVA_eucl))
print('Test CRPS: {}'.format(CRPS_FSVA_eucl))
print('Test LS: {}'.format(LS_FSVA_eucl))
print('Runtime: {} seconds'.format(runtime_FSVA_eucl))

Test MAE: 0.27599135140405523
Test RMSE: 0.3631069645635602
Test CRPS: 0.2330028896455295
Test LS: 2.7943730093335666
Runtime: 1458.7386960983276 seconds
